In [3]:
#!pip install ipython-sql # https://github.com/catherinedevlin/ipython-sql

In [4]:
%load_ext sql

In [5]:
%sql sqlite://

'Connected: None@None'

In [12]:
%%sql
#DROP TABLE donuts
CREATE TABLE donuts (doNum, dough, glaze, filling);
INSERT INTO donuts VALUES (1, 'cake', 'maple', 'none');
INSERT INTO donuts VALUES (2, 'yeast', 'sugar', 'none');

Done.
1 rows affected.
1 rows affected.


[]

In [13]:
%sql select * from donuts

Done.


doNum,dough,glaze,filling
1,cake,maple,none
2,yeast,sugar,none


In [14]:
%%sql
INSERT INTO donuts VALUES (3, 'yeast', 'maple', 'boston creme');
INSERT INTO donuts VALUES (4, 'yeast', 'chocolate', 'none');
INSERT INTO donuts VALUES (5, 'cruller', 'vanilla', 'none');
INSERT INTO donuts VALUES (6, 'yeast', 'chocolate ', 'boston creme');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [15]:
%sql select * from donuts

Done.


doNum,dough,glaze,filling
1,cake,maple,none
2,yeast,sugar,none
3,yeast,maple,boston creme
4,yeast,chocolate,none
5,cruller,vanilla,none
6,yeast,chocolate,boston creme
